https://vgpena.github.io/classifying-tweets-with-keras-and-tensorflow/

En el anterior enlace, tenéis un ejemplo sobre cómo, a partir de tweets con un label específico (un sentimiento, positivo o negativo): 

1. Genera un conjunto de entrenamiento. El conjunto de entrenamiento es formado a partir de tweets completos pasados a un array con un tamaño específico.
2. Ese array (X_train de tamaño N) tiene un label que representa el sentimiento (y_train)
3. Como todas las frases tienen un tamaño N, la entrada de la red neuronal será de tamaño N y la salida de la red será de tamaño 2 usando activación softmax(porque hay dos clases).

Se pide: 

- Realizar un clasificador de reviews para el dataset de IMDB de la carpeta data_exercise/

**Cuando usa la importación "keras.x", reemplázalo por "tensorflow.keras.x"**

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [13]:
imdb_archive = pd.read_csv("./data_exercise/IMDBDataset.csv")

In [21]:
imdb_archive.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [30]:
X_train = [x[0] for x in imdb_archive.values]
y_train = [x[1] for x in imdb_archive.values]

In [31]:
max_words = 3000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(X_train)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index

def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in tf.keras.preprocessing.text.text_to_word_sequence(text)]

allWordIndices = []
# Change each token to its ID in the Tokenizer's word_index
for text in X_train:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

X_train = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
onehot = pd.get_dummies(imdb_archive.sentiment)
target_labels = onehot.columns
y_train = onehot.values

In [32]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(512, input_shape=(max_words,), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation='sigmoid'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [33]:
model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train,
    batch_size=32,
    epochs=5,
    verbose=1,
    validation_split=0.2,
    shuffle=True)

Epoch 1/5
1250/1250 [==============================] - 12s 9ms/step - loss: 0.3362 - accuracy: 0.8533 - val_loss: 0.2752 - val_accuracy: 0.8817
Epoch 2/5
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2475 - accuracy: 0.8988 - val_loss: 0.2818 - val_accuracy: 0.8816
Epoch 3/5
1250/1250 [==============================] - 13s 10ms/step - loss: 0.1960 - accuracy: 0.9169 - val_loss: 0.2913 - val_accuracy: 0.8802
Epoch 4/5
1250/1250 [==============================] - 14s 11ms/step - loss: 0.1232 - accuracy: 0.9486 - val_loss: 0.3497 - val_accuracy: 0.8813
Epoch 5/5
1250/1250 [==============================] - 14s 11ms/step - loss: 0.0752 - accuracy: 0.9693 - val_loss: 0.4126 - val_accuracy: 0.8827
